In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage.filters import (
    threshold_multiotsu, threshold_isodata, threshold_li,
    threshold_mean, threshold_minimum, threshold_otsu,
    threshold_triangle, threshold_yen
)

# Define the main folder path
main_folder_path = 'D:/beamstop/20241002'

# Specify the target filename (e.g., "target_file.npz")
target_filename = "cbf2stat_max.npz"

# Define the thresholding methods
thresholding_methods = [
    ('Isodata', threshold_isodata),
    ('Li', threshold_li),
    ('Mean', threshold_mean),
    ('Minimum', threshold_minimum),
    ('Otsu', threshold_otsu),
    ('Triangle', threshold_triangle),
    ('Yen', threshold_yen)
]

def process_images(folder_range):
    # Determine the start and end of the range
    if len(folder_range) == 1:
        start = folder_range[0]
        end = start
    elif len(folder_range) == 2:
        start, end = folder_range
    else:
        raise ValueError("Input must be a tuple of (start,) or (start, end)")

    # Iterate over the folder range specified
    for folder_num in range(start, end + 1):
        folder_name = f"{folder_num:03d}"  # Format folder number with leading zeros (e.g., 001, 002)
        folder_path = os.path.join(main_folder_path, folder_name)

        # Check if the folder exists
        if not os.path.exists(folder_path):
            print(f"Folder {folder_name} does not exist.")
            continue

        # Check if the target file exists in the folder
        npz_path = os.path.join(folder_path, target_filename)
        if not os.path.exists(npz_path):
            print(f"File {target_filename} not found in folder {folder_name}.")
            continue

        # Load the .npz file
        data = np.load(npz_path)

        # Assuming the desired image is stored with the key 'image' in the .npz file
        if 'image' not in data:
            print(f"'image' key not found in {target_filename} in folder {folder_name}.")
            continue

        max_image = data['image']

        # Remove all values that are -1 or -2
        filtered_image = max_image[(max_image != -1) & (max_image != -2)]

        # Apply clipping to the filtered image
        p1, p2 = np.percentile(filtered_image[filtered_image > 0], (1, 98))
        clipped_image = np.clip(max_image, p1, p2)

        # Normalize the clipped image to [0, 255]
        clipped_normalized_image = (clipped_image - clipped_image.min()) / (clipped_image.max() - clipped_image.min()) * 255
        clipped_normalized_image = clipped_normalized_image.astype(np.uint8)

        # Plot the thresholding results
        fig, axs = plt.subplots(2, 4, figsize=(16, 8))
        axs = axs.flat

        for j, (name, method) in enumerate(thresholding_methods):
            try:
                # Calculate threshold value using the given method
                thresh_value = method(clipped_normalized_image)
                mask = clipped_normalized_image > thresh_value

                # Plot the mask created by thresholding
                axs[j].imshow(mask, cmap='gray', aspect='equal')
                axs[j].set_title(f'{name} Threshold for Folder {folder_name} (Value: {thresh_value:.2f})')
                axs[j].axis('off')
                axs[j].grid(False)

            except RuntimeError as e:
                # Handle methods that may fail to find a valid threshold value
                print(f"Thresholding method '{name}' failed for Folder {folder_name} with error: {e}")
                axs[j].set_title(f'{name} Failed')
                axs[j].axis('off')
                axs[j].grid(False)
            except ValueError as e:
                # Handle any other issues with processing the image
                print(f"Thresholding method '{name}' raised ValueError for Folder {folder_name} with error: {e}")
                axs[j].set_title(f'{name} Failed')
                axs[j].axis('off')
                axs[j].grid(False)

        # Plot the original clipped and normalized image
        axs[-1].imshow(clipped_normalized_image, cmap='gray', aspect='equal')
        axs[-1].set_title(f'Original Clipped Image: Folder {folder_name}')
        axs[-1].axis('off')
        axs[-1].grid(False)

        plt.tight_layout()
        plt.show()


process_images((33,))  
